# Running EVcouplings complex pipeline jobs

## Content

This notebook serves as a supplement to the running_jobs notebook, and describes stages and parameters that differ between the monomer and complexes pipeline. 

------

# Pipeline Stages

The evcouplings pipeline is split into different stages for different aspects of computation. This allows users to run only the desired stages, and to re-start computation from an arbitrary stage. The stages are as follows:

__align\_1 and align\_2__: Creates or reads in an existing alignment, and pre-processes alignment for couplings calculation. There is one stage for each monomer alignment. 

__concatenate__: Pairs up the sequences in the monomer alignments to create a concatenated alignment.  

__couplings__: calculates ECs on a concatenated alignment and renumbers the ECs according to the input monomers. 

__compare__: Searches for structures against which to compare the couplings. First identifies structures for comparison for each monomer independently, then identifies heteromultimeric structures for comparison by taking the intersection of these two lists of structures. 

__mutate__: Not yet available for complexes pipeline. 

__fold__: Not yet available for complexes pipeline. 


------

# Configuration files

The config file drives all aspects of the pipeline and is required for every job. The config files are in computable YAML format (see below for an example of how to modify it programmatically).

> For an example configuration file, see config/sample_config_complex.txt in this repository.

### Parameters that differ between monomer and complexes pipeline

Please see the running_jobs.ipynb in this directory for an explanation of how to set up your configuration file. The following parameters are the most important ones that differ between the two pipelines. 

1) In "align_1" and "align_2" section:
* __alignment_protocol__: choose either 'existing' to use an input alignment or 'standard' to generate an alignment
using the monomer alignment protocol.
* __input_alignment__: input alignment file, required for 'existing' alignment protocol
* __override_annotation_file__: input annotation file, required for 'existing' alignment protocol when using best_hit concatenation. This is the \_annotations.csv file from the same monomer pipeline run used to generate the input alignment, OR a user-generated .csv file with columns "id" containing the full sequence ids from the alignment and column "OS" that contains annotation information for each id. This will override the incomplete annotations generated when postprocessing the input_alignment.

3) in "concatenate" section:
* __protocol__: currently two protocols are available. Genome_distance will pair sequences by closest reciprocal distance on the genome. Best_hit will pair by best hit to the target sequence for each genome. 
* __use_best_reciprocal__: if using the best_hit protocol, use_best_reciprocal specifies whether to only take the best reciprocal hit in each genome
* __minimum_sequence_coverage__: After concatenation, only keep concatenated sequences that align to at least x% of the target concatenated sequence 
* __minimum_column_coverage__: After concatenation, only include alignment columns with at least x% residues (rather than gaps) during model inference. 

4) in "couplings" section:
* __scoring__: Options are skewnormal, lognormal, and evcomplex. Scoring model to assess confidence in computed ECs
* __use_all_ecs_for_scoring__: if True, will run the scoring model on the ECs, both inter and intra, simultaneously. If false, scoring will be done for monomer 1, monomer 2, and inter-protein ECs independently. 

5) in "compare" section:

Arguments in this section echo the arguments in the "compare" section of the monomer pipeline, with the prefix "first" applying to the first monomer, and with prefix "second" applying to second monomer. Arguments for each monomer can be set independently in the config file. 



### Batch jobs

Currently not available for complexes pipeline

### Modifying the config file from within Python:

Can be modified using the same syntax as monomer config files. 

------

# Running the pipeline

The complexes pipeline can be submitted in the same three ways (via python, evcouplings_runcfg, or evcouplings) as the monomer pipeline. 

Currently, the evcouplings application __does not support__ modifying the arguments of the complexes pipeline via command line. This will be supported in future releases. 